In [16]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [17]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
df = pd.read_csv("simulated_feedback_1500_v2.csv")

In [19]:
df.head()

,Text,Sentiment,Source,Date/Time,User ID,Location,Confidence Score,Product/Service,Category,Rating,Response Time (mins),Resolution Status
0,"It was okay, nothing special.",Neutral,Website Testimonial,2023-08-29 06:04:00,user10000,Toronto,0.78,Website,Customer Support,4,152,Resolved
1,Received wrong item.,Negative,Amazon Reviews,2023-06-01 09:31:00,user10001,Toronto,0.88,Mobile App,Delivery,5,133,Resolved
2,Product quality is acceptable.,Neutral,Website Testimonial,2023-09-16 21:12:00,user10002,London,0.76,Gift Card,Usability,5,103,Resolved
3,Slow response from team.,Negative,Yelp,2023-08-26 02:44:00,user10003,Berlin,0.74,Website,Billing,3,134,Escalated
4,Great quality and quick delivery.,Positive,TripAdvisor,2023-06-23 16:11:00,user10004,Paris,0.71,Delivery Service,Billing,3,165,Resolved


In [20]:
df.shape

(1500, 12)

In [21]:
df.drop_duplicates(inplace=True)

In [22]:
df.dropna(subset=['Text', 'Sentiment'], inplace=True)

In [23]:
def normalize(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['Cleaned_Text'] = df['Text'].apply(normalize)

In [24]:
df.shape

(1500, 13)

In [25]:
Stops = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

def refine_text(sentence):
    sentence = str(sentence)
    words = word_tokenize(sentence)
    filtered = [w for w in words if w.lower() not in Stops]
    lemmatized = [lemma.lemmatize(w.lower()) for w in filtered]
    return ' '.join(lemmatized)

df['Processed_Text'] = df['Cleaned_Text'].apply(refine_text)

In [27]:
df['Source'] = df['Source'].fillna('Unknown')
df['Category'] = df['Category'].fillna('General')
df['Rating'] = df['Rating'].fillna(df['Rating'].median())

df = df[df['Processed_Text'].str.split().str.len() >= 3]

df['Sentiment'] = np.where(df['Rating'] > 3, 'Positive',
                   np.where(df['Rating'] == 3, 'Neutral', 'Negative'))

In [28]:
df.to_csv("cleaned_dataset.csv", index=False)

In [29]:
df.shape

(1161, 14)